In [ ]:
import numpy as np
import pandas as pd
from linearmodels.system import SUR

# Simulasi data sesuai data yang disediakan oleh user
data = pd.read_excel("D:/E/PENS/Semester5/Ekonometrika Terapan/Data_ekpor_impor_USD.xlsx")

# Buat dataframe dari data
df = pd.DataFrame(data)

# Definisikan persamaan SUR
equations = {
    "Total_Ekspor": "Total_Ekspor ~ IDR_to_USD + MIGAS + NON_MIGAS",
    "NON_MIGAS": "NON_MIGAS ~ Agriculture + Industry + Mining + Others + IDR_to_USD",
    "Total_Impor": "Total_Impor ~ Consumption_Goods + Raw_Material_Support + Capital_Goods + IDR_to_USD",
    "Raw_Material_Support": "Raw_Material_Support ~ Consumption_Goods + Capital_Goods + IDR_to_USD"
}

# Buat dan fit model SUR
model = SUR.from_formula(equations, df)
result = model.fit()

# Cetak ringkasan hasil
print(result.summary)


                           System GLS Estimation Summary                           
Estimator:                        GLS   Overall R-squared:                   0.9984
No. Equations.:                     4   McElroy's R-squared:                 1.0000
No. Observations:                  69   Judge's (OLS) R-squared:             0.9664
Date:                Sun, Nov 17 2024   Berndt's R-squared:                  1.0000
Time:                        14:33:20   Dhrymes's R-squared:                 0.9986
                                        Cov. Estimator:                      robust
                                        Num. Constraints:                      None
           Equation: Total_Ekspor, Dependent Variable: Total_Ekspor           
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
IDR_to_USD -9.474e-08  1.686e-07    -0.5619     0.5742  -4.252e-07   2.357e-07
MIGAS       

In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Buat dataframe hanya dengan variabel independen
independent_vars = df[['IDR_to_USD', 'MIGAS', 'NON_MIGAS', 'Agriculture', 'Industry', 'Mining', 'Others', 'Consumption_Goods', 'Raw_Material_Support', 'Capital_Goods']]

# Hitung VIF untuk setiap variabel independen
vif_data = pd.DataFrame()
vif_data['Variable'] = independent_vars.columns
vif_data['VIF'] = [variance_inflation_factor(independent_vars.values, i) for i in range(independent_vars.shape[1])]

print(vif_data)


               Variable           VIF
0            IDR_to_USD  5.776144e+01
1                 MIGAS  2.517011e+10
2             NON_MIGAS  6.541176e+12
3           Agriculture  2.695965e+09
4              Industry  4.181615e+12
5                Mining  3.966531e+11
6                Others  1.108390e+05
7     Consumption_Goods  9.125056e+01
8  Raw_Material_Support  2.356747e+02
9         Capital_Goods  1.127650e+02


UJi HETEROSKEDASTISITAS

In [23]:
from statsmodels.stats.diagnostic import het_breuschpagan
import statsmodels.api as sm

# Ambil residual dari persamaan Total_Ekspor
residuals = result.resids['Total_Ekspor']

# Tambahkan konstanta ke variabel independen dalam persamaan
exog = sm.add_constant(df[['IDR_to_USD', 'MIGAS', 'NON_MIGAS']])

# Lakukan uji Breusch-Pagan
bp_test = het_breuschpagan(residuals, exog)
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))


{'Lagrange multiplier statistic': np.float64(1.4598072004505729), 'p-value': np.float64(0.6915773824764916), 'f-value': np.float64(0.4683012395838666), 'f p-value': np.float64(0.7054079149544173)}


 Uji Autokorelas

In [24]:
from statsmodels.stats.stattools import durbin_watson

# Hitung Durbin-Watson untuk residual setiap persamaan
for eq_name in result.resids:
    dw_stat = durbin_watson(result.resids[eq_name])
    print(f'Durbin-Watson stat for {eq_name}: {dw_stat}')


Durbin-Watson stat for Total_Ekspor: 2.029327033760584
Durbin-Watson stat for NON_MIGAS: 1.389778815369028
Durbin-Watson stat for Total_Impor: 2.3433642340138303
Durbin-Watson stat for Raw_Material_Support: 0.3863043828299623


4. Uji Normalitas Residual

In [25]:
from statsmodels.stats.stattools import jarque_bera

# Melakukan uji Jarque-Bera untuk setiap persamaan dalam model SUR
for eq_name in result.resids:
    jb_stat, jb_pvalue, skew, kurtosis = jarque_bera(result.resids[eq_name])
    print(f'Jarque-Bera stat for {eq_name}: {jb_stat}, p-value: {jb_pvalue}')


Jarque-Bera stat for Total_Ekspor: 2.6913974832191907, p-value: 0.26035772151326286
Jarque-Bera stat for NON_MIGAS: 2.598666021286648, p-value: 0.27271362947406586
Jarque-Bera stat for Total_Impor: 3.012910063791412, p-value: 0.2216944864924649
Jarque-Bera stat for Raw_Material_Support: 7.210898062982843, p-value: 0.02717523953464324


### Korelasi residual dari setiap persamaan

In [26]:
import numpy as np
from scipy.stats import chi2

# Ambil matriks kovarian residual dari hasil model SUR
cov_matrix = result.sigma  # Matriks kovarian residual
n_obs = result.nobs  # Jumlah observasi
equation_labels = result.equation_labels  # Label persamaan dalam sistem
m_eq = len(equation_labels)  # Jumlah persamaan dalam sistem

# Hitung statistik Breusch-Pagan
bp_stat = 0
for i in range(m_eq):
    for j in range(i + 1, m_eq):
        eq_i = equation_labels[i]  # Label persamaan i
        eq_j = equation_labels[j]  # Label persamaan j

        # Elemen non-diagonal
        cov_ij = cov_matrix.loc[eq_i, eq_j]  # Kovarian residual antar persamaan
        var_i = cov_matrix.loc[eq_i, eq_i]  # Variansi residual persamaan i
        var_j = cov_matrix.loc[eq_j, eq_j]  # Variansi residual persamaan j

        # Korelasi residual
        rho_ij = cov_ij / np.sqrt(var_i * var_j)
        bp_stat += n_obs * rho_ij**2

# Derajat kebebasan
df = int(m_eq * (m_eq - 1) / 2)

# Nilai p-value
p_value = 1 - chi2.cdf(bp_stat, df)

# Hasil uji BP
print("Hasil Uji Breusch-Pagan:")
print(f"Breusch-Pagan Statistic: {bp_stat:.4f}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value:.4f}")

# Interpretasi
if p_value < 0.05:
    print("Terdapat korelasi residual antar persamaan (tolak H0).")
else:
    print("Tidak terdapat korelasi residual antar persamaan (gagal tolak H0).")


Hasil Uji Breusch-Pagan:
Breusch-Pagan Statistic: 16.2680
Degrees of Freedom: 6
P-value: 0.0124
Terdapat korelasi residual antar persamaan (tolak H0).
